# The Graph Data Access

In this notebook, we retrieve live Rai system data from [The Graph](https://thegraph.com/explorer/subgraph/reflexer-labs/rai-mainnet?query=Example%20query) and save it as a CSV for analysis in the [Graph Analysis](TheGraphDataAnalysis.ipynb) notebook.

## Full Reflexer Labs dashboad query
```
{
  internalCoinBalances(where: {accountHandler: "${GEB_STABILITY_FEE_TREASURY}"}) { 
    balance
  }
  accountingEngine(id: "current") { # can it use block number?
    activeDebtAuctions
    activeSurplusAuctions
  }
  collateralType(id: "ETH-A") {
    currentPrice {
      value
    }
    currentMedianizerUpdate {
      value
    }
    debtCeiling
    activeLiquidations
    liquidationsStarted
    totalCollateral
    totalAnnualizedStabilityFee
  }
  systemState(id: "current") { # can it use block number?
    coinUniswapPair {
      reserve0
      reserve1
    }
    currentCoinMedianizerUpdate{
      value
    }
    currentRedemptionRate {
      eightHourlyRate
      annualizedRate
      hourlyRate
      createdAt
    }
    currentRedemptionPrice {
      value
    }
    erc20CoinTotalSupply
    globalDebt
    globalDebtCeiling
    totalActiveSafeCount
    coinAddress
    wethAddress
    systemSurplus
    debtAvailableToSettle
  }
  dailyStats(first: 1000, orderDirection: desc) {
    globalDebt
    timestamp
    redemptionRate {
      twentyFourHourlyRate
    }
  }
  hourlyStats(first: 1000, orderBy: timestamp, orderDirection: desc) {
    globalDebt
    timestamp
    redemptionRate {
      hourlyRate
    }
    redemptionPrice{
      value
    }
    marketPriceUsd
  }
}
```

In [1]:
# import libraries
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt

# Graph
url = 'https://api.thegraph.com/subgraphs/name/reflexer-labs/rai-mainnet'

## Hourly stats

In [3]:
query_header = '''
query {{
    hourlyStats(first: 1000, skip:{}) {{'''

query_tail = '''    
}
}'''

query_body = '''
 timestamp
    blockNumber
    redemptionRate {
      hourlyRate
    }
    redemptionPrice {
      id
      block
      value
      
    }
    marketPriceUsd # price of COIN in USD (uni pool price * ETH median price)
    marketPriceEth # Price of COIN in ETH (uni pool price)
    globalDebt
    erc20CoinTotalSupply
'''


n = 0
hourly = []
while True:
    print(f'request {n+1}')
    query = query_header.format(n*1000) + query_body + query_tail
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['hourlyStats']
    print(f'results {len(s)}')
    hourly.extend(s)
    print(f'total {len(hourly)}')
    n += 1
    if len(s) < 1000:
        break
        
hourlyStats = pd.DataFrame(hourly)

request 1
results 834
total 834


In [5]:
hourlyStats

,blockNumber,erc20CoinTotalSupply,globalDebt,marketPriceEth,marketPriceUsd,redemptionPrice,redemptionRate,timestamp
0,11857308,1988.440432971667032725,1988.577499489843641169436691184972,0.002589462210048543650949087456484599,4.670500166535429740591356399938391,"{'block': '11849022', 'id': '0xe313271188492c4...",{'hourlyRate': '1'},1613338681
1,11860793,2788.440432971667032725,2788.617717345781163523077653358556,0.002481355746535171105569862943787003,4.377049344386119132419461918241039,"{'block': '11860755', 'id': '0x40066774f6fb7d3...",{'hourlyRate': '1'},1613384616
2,11861040,2788.440432971667032725,2788.657935201718685876567866781341,0.002481355746535171105569862943787003,4.406833024856393396658172078078713,"{'block': '11861008', 'id': '0x0d551e2a2d55fa8...",{'hourlyRate': '1'},1613387890
3,11861337,2788.440432971667032725,2788.657935201718685876567866781341,0.002481355746535171105569862943787003,4.419265710853707004204445293455575,"{'block': '11861237', 'id': '0x5c67dbb323db039...",{'hourlyRate': '1'},1613392101
4,11861474,3048.440432971667032725,3048.645954953510904626288422086589,0.001768934955701819449591545208406114,3.194329891923853932595347583210722,"{'block': '11861237', 'id': '0x5c67dbb323db039...",{'hourlyRate': '1'},1613394096
...,...,...,...,...,...,...,...,...
829,12089344,43487860.667051349456944081,43734759.10991572746971733487829186,0.00166190689134055979943670720138926,2.961266118000318663788795713922176,"{'block': '12089330', 'id': '0xc10a4cb762e1703...",{'hourlyRate': '1.000022024491477833300653081'},1616426158
830,12089714,43511896.036432746638848263,43758790.10435062289909417424501354,0.001662919533436415830937551013666968,2.957407846722545124011695620756376,"{'block': '12089617', 'id': '0xb07aac45d52cb9c...",{'hourlyRate': '1.000022024491477833300653081'},1616430475
831,12089899,43403731.392585009614677379,43650842.40166812414301912924501354,0.001675952194066927327638159924307098,2.968899033223739753091171161112295,"{'block': '12089617', 'id': '0xb07aac45d52cb9c...",{'hourlyRate': '1.000022024491477833300653081'},1616433066
832,12090203,43390331.392585009614677379,43637469.27747070418925089924501354,0.001673900347181184118392335848904411,2.983476283798383513416579800294777,"{'block': '12090152', 'id': '0xee81ea069ecd53b...",{'hourlyRate': '1.000022024491477833300653081'},1616437077


In [34]:
hourlyStats.redemptionPrice.values[-1]

{'block': '12090480',
 'id': '0x128e8125b2cc0ff141b8ce58b2f47c70509c7a524efff7c565490dcb59c3b7d7-240',
 'redemptionRate': '1.000000006215034036996657478',
 'value': '3.020087944526902135786975966'}

In [35]:
hourlyStats.redemptionRate.values[-1]

{'hourlyRate': '1.000022374372766204773206208'}

In [7]:
hourlyStats['redemptionPriceActual'] = hourlyStats.redemptionPrice.apply(lambda x: x['value'])

In [8]:
hourlyStats['redemptionRateActual'] = hourlyStats.redemptionPrice.apply(lambda x: x['redemptionRate'])

In [9]:
hourlyStats['erc20CoinTotalSupply'] = hourlyStats['erc20CoinTotalSupply'].astype(float)
hourlyStats['globalDebt'] = hourlyStats['globalDebt'].astype(float)
hourlyStats['blockNumber'] = hourlyStats['blockNumber'].astype(int)
hourlyStats['marketPriceEth'] = hourlyStats['marketPriceEth'].astype(float)
hourlyStats['marketPriceUsd'] = hourlyStats['marketPriceUsd'].astype(float)
hourlyStats['redemptionPriceActual'] = hourlyStats['redemptionPriceActual'].astype(float)
hourlyStats['redemptionRateActual'] = hourlyStats['redemptionRateActual'].astype(float)

In [10]:
hourlyStats.describe()

,blockNumber,erc20CoinTotalSupply,globalDebt,marketPriceEth,marketPriceUsd,redemptionPriceActual,redemptionRateActual
count,8.340000e+02,8.340000e+02,8.340000e+02,834.000000,834.000000,834.000000,8.340000e+02
mean,1.197470e+07,3.305505e+07,3.350672e+07,0.001885,3.223918,3.066364,1.000000e+00
std,6.609387e+04,1.428475e+07,1.441385e+07,0.000302,0.447405,0.049342,1.633703e-08
min,1.185731e+07,1.988440e+03,1.988577e+03,0.001600,2.850455,3.013686,9.999999e-01
25%,1.191771e+07,2.714650e+07,2.737521e+07,0.001668,3.014544,3.017765,1.000000e+00
50%,1.197444e+07,3.449464e+07,3.546279e+07,0.001784,3.151019,3.049511,1.000000e+00
75%,1.203184e+07,4.553056e+07,4.571930e+07,0.002057,3.272902,3.119491,1.000000e+00
max,1.209052e+07,4.899876e+07,4.918825e+07,0.004474,8.055382,3.140000,1.000000e+00


In [11]:
hourlyStats.to_csv('saved_results/RaiLiveGraphData.csv')

## Daily stats - WIP

In [13]:
query_header = '''
query {{
    dailyStats(first: 1000, skip:{}) {{'''

query_tail = '''    
}
}'''

query_body = '''
    timestamp
    globalDebt
    redemptionRate {
      twentyFourHourlyRate
    }
'''



n = 0
results = []
while True:
    print(f'request {n+1}')
    query = query_header.format(n*1000) + query_body + query_tail
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['dailyStats']
    print(f'results {len(s)}')
    results.extend(s)
    print(f'total {len(results)}')
    n += 1
    if len(s) < 1000:
        break
        
dailyStats = pd.DataFrame(results)

dailyStats.head()

request 1
results 37
total 37


,globalDebt,redemptionRate,timestamp
0,1988.508966230755336947218345592486,{'twentyFourHourlyRate': '1'},1613338681
1,2788.522357955470745187436691184972,{'twentyFourHourlyRate': '1'},1613383785
2,7204.841129160410399042582689488635,{'twentyFourHourlyRate': '1'},1613436497
3,10603.09390239356752028390089696083,{'twentyFourHourlyRate': '0.999999085220771157...,1613520522
4,4755099.686882616278066513280682631,{'twentyFourHourlyRate': '0.999999911975837683...,1613606864


## System State - WIP

In [22]:
blocknumbers = hourlyStats.blockNumber.values.tolist()
state = []
for i in blocknumbers:
    query = '''
    {
      systemState(block: {number:%s},id:"current") { 
        coinUniswapPair {
          reserve0
          reserve1
        }
        currentCoinMedianizerUpdate{
          value
        }
        currentRedemptionRate {
          eightHourlyRate
          annualizedRate
          hourlyRate
          createdAt
        }
        currentRedemptionPrice {
          value
        }
        erc20CoinTotalSupply
        globalDebt
        globalDebtCeiling
        totalActiveSafeCount
        coinAddress
        wethAddress
        systemSurplus
        debtAvailableToSettle
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['systemState']
    state.append(s)
        
systemState = pd.DataFrame(state)

systemState.head()
    

,coinAddress,coinUniswapPair,currentCoinMedianizerUpdate,currentRedemptionPrice,currentRedemptionRate,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,systemSurplus,totalActiveSafeCount,wethAddress
0,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '661.764842786150958141', 'reserv...",None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,2788.440432971667032725,2788.522357955470745187436691184972,115792089237316195423570985008687.9,0.06853325908830422221834559248594176,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
1,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '676.933726709008306816', 'reserv...",None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,2788.440432971667032725,2788.657935201718685876567866781341,115792089237316195423570985008687.9,0.1087511150258265757085590152709335,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
2,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '676.933726709008306816', 'reserv...",None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,2788.440432971667032725,2788.657935201718685876567866781341,115792089237316195423570985008687.9,0.1087511150258265757085590152709335,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
3,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '676.933726709008306816', 'reserv...",None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,2788.440432971667032725,2788.657935201718685876567866781341,115792089237316195423570985008687.9,0.1087511150258265757085590152709335,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
4,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '801.933726709008306816', 'reserv...",{'value': '0'},{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,3048.440432971667032725,3048.645954953510904626288422086589,115792089237316195423570985008687.9,0.1161617985274885650688366678947734,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2


## Internal Coin Balances - WIP

balance = []
for i in blocknumbers:
    query = '''
    {
      internalCoinBalances((block: {number:%s},where: {accountHandler: "${GEB_STABILITY_FEE_TREASURY}"}) {
        balance
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['internalCoinBalances']
    balance.append(s)
        
coinBalances = pd.DataFrame(balance)

coinBalances.head()


In [27]:
r.content

b'{"errors":[{"locations":[{"column":28,"line":3}],"message":"Unexpected `([Punctuator]`\\nExpected `Name`"}]}'

## Accounting Engine - WIP

In [31]:
blocknumbers = hourlyStats.blockNumber.values.tolist()
accounts = []
for i in blocknumbers:
    query = '''
    {
      accountingEngine(block: {number:%s},id: "current") { 
        activeDebtAuctions
        activeSurplusAuctions
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['accountingEngine']
    accounts.append(s)
        
accountingEngine = pd.DataFrame(accounts)

accountingEngine.head()
        

blocknumber 11857308
blocknumber 11860793
blocknumber 11861040
blocknumber 11861337
blocknumber 11861474
blocknumber 11861791
blocknumber 11862113
blocknumber 11862329
blocknumber 11862624
blocknumber 11862906
blocknumber 11863227
blocknumber 11863455
blocknumber 11863774
blocknumber 11864026
blocknumber 11864260
blocknumber 11864895
blocknumber 11865064
blocknumber 11865513
blocknumber 11865735
blocknumber 11865824
blocknumber 11866378
blocknumber 11866683
blocknumber 11867014
blocknumber 11867211
blocknumber 11867503
blocknumber 11867818
blocknumber 11868157
blocknumber 11868417
blocknumber 11868704
blocknumber 11868901
blocknumber 11869333
blocknumber 11869538
blocknumber 11869788
blocknumber 11870145
blocknumber 11870443
blocknumber 11870767
blocknumber 11870961
blocknumber 11871228
blocknumber 11871495
blocknumber 11871769
blocknumber 11872047
blocknumber 11872406
blocknumber 11872744
blocknumber 11872943
blocknumber 11873231
blocknumber 11873560
blocknumber 11873687
blocknumber 1

blocknumber 11967817
blocknumber 11967928
blocknumber 11968237
blocknumber 11968564
blocknumber 11968695
blocknumber 11968993
blocknumber 11969259
blocknumber 11969471
blocknumber 11969791
blocknumber 11970092
blocknumber 11970339
blocknumber 11970629
blocknumber 11970958
blocknumber 11971227
blocknumber 11971428
blocknumber 11971736
blocknumber 11971995
blocknumber 11972384
blocknumber 11972558
blocknumber 11972870
blocknumber 11973210
blocknumber 11973486
blocknumber 11973599
blocknumber 11973991
blocknumber 11974303
blocknumber 11974578
blocknumber 11974779
blocknumber 11974960
blocknumber 11975202
blocknumber 11975591
blocknumber 11975734
blocknumber 11976137
blocknumber 11976314
blocknumber 11976604
blocknumber 11976874
blocknumber 11977150
blocknumber 11977435
blocknumber 11977701
blocknumber 11977992
blocknumber 11978248
blocknumber 11978553
blocknumber 11978776
blocknumber 11979063
blocknumber 11979317
blocknumber 11979581
blocknumber 11979861
blocknumber 11980224
blocknumber 1

blocknumber 12076685
blocknumber 12076973
blocknumber 12077308
blocknumber 12077459
blocknumber 12077877
blocknumber 12078035
blocknumber 12078274
blocknumber 12078700
blocknumber 12079162
blocknumber 12079629
blocknumber 12079684
blocknumber 12079947
blocknumber 12080197
blocknumber 12080531
blocknumber 12080819
blocknumber 12081066
blocknumber 12081458
blocknumber 12081665
blocknumber 12081842
blocknumber 12082014
blocknumber 12082319
blocknumber 12082576
blocknumber 12082829
blocknumber 12083160
blocknumber 12083478
blocknumber 12083790
blocknumber 12083961
blocknumber 12084283
blocknumber 12084584
blocknumber 12084916
blocknumber 12085111
blocknumber 12085404
blocknumber 12085546
blocknumber 12086012
blocknumber 12086165
blocknumber 12086361
blocknumber 12086624
blocknumber 12086932
blocknumber 12087255
blocknumber 12087567
blocknumber 12087684
blocknumber 12087979
blocknumber 12088259
blocknumber 12088584
blocknumber 12089054
blocknumber 12089344
blocknumber 12089714
blocknumber 1

,activeDebtAuctions,activeSurplusAuctions
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


## Eth Price

In [33]:
blocknumbers = hourlyStats.blockNumber.values.tolist()
eth_price = []
for i in blocknumbers:
    query = '''
    {
      collateralType(block: {number:%s}, id: "ETH-A") {
        currentFsmUpdate {
          value
        }
        currentMedianizerUpdate {
          value
        }
        debtCeiling
        activeLiquidations
        liquidationsStarted
        totalCollateral
        totalAnnualizedStabilityFee
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['collateralType']
    eth_price.append(s)
        
collateralType = pd.DataFrame(eth_price)

collateralType.head()
    

,activeLiquidations,currentFsmUpdate,currentMedianizerUpdate,debtCeiling,liquidationsStarted,totalAnnualizedStabilityFee,totalCollateral
0,0,{'value': '1803.65643044'},{'value': '1803.65643044'},10000000,0,1.019999999999999999972831879,14.892360473369843164
1,0,{'value': '1803.65643044'},{'value': '1763.97493608'},10000000,0,1.019999999999999999972831879,14.892360473369843164
2,0,{'value': '1763.97493608'},{'value': '1775.97792296'},10000000,0,1.019999999999999999972831879,14.892360473369843164
3,0,{'value': '1763.97493608'},{'value': '1780.98836373'},10000000,0,1.019999999999999999972831879,14.892360473369843164
4,0,{'value': '1763.97493608'},{'value': '1805.79273513'},10000000,0,1.019999999999999999972831879,16.892360473369843164


In [ ]:
collateralType

## Safes

## Conclusion

Using The Graph, a lot of data about the Rai system can be obtained for analyzing the health of the system. With some data manipulation, these data streams could be intergrated into the Rai cadCAD model to turn it into a true decision support system.